In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, trange
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

In [2]:
portfolio_tickers = ["NVDA","WRK", "AAPL"]
start = "2023-01-30"
end = "2023-10-30"
Interval = "1d"
rawData = yf.download(tickers=portfolio_tickers,start=start, end =end, interval =Interval )

[*********************100%%**********************]  3 of 3 completed


close_data_AAPL = rawData["Close"]["AAPL"]
close_data_AAPL_Frame_y = close_data_AAPL.to_frame()
close_data_AAPL_Frame_x = close_data_AAPL_Frame_y.shift(-1)
x_train = close_data_AAPL_Frame_x.iloc[:150]
x_test = close_data_AAPL_Frame_x.iloc[150:]
y_train = close_data_AAPL_Frame_y.iloc[:150]
y_test = close_data_AAPL_Frame_y.iloc[150:]
x_train_numpy = x_train.to_numpy()
y_train_numpy = y_train.to_numpy()
model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=1)
#model.fit(x_train_numpy, y_train_numpy)

In [4]:
close_data_AAPL[4]

NameError: name 'close_data_AAPL' is not defined

In [167]:
def Model(PriceNow, Price1Day, Price2Day, Price3Day):
    if PriceNow > Price1Day and Price1Day > Price2Day and Price2Day > Price3Day:
        position_size = 3
        return True, position_size

In [5]:
def BackTest(Model):
    close_data_AAPL.plot()
    plt.ylabel("Price")
    portfolio_trades = []
    portfolio_Balance = 0
    portfolio_Balance_OT = []
    position = 0
    profit_and_loss = 0
    profit_and_loss_graph = []


    for day in range(5, close_data_AAPL.size):
        buy, position_size = Model(close_data_AAPL[day], close_data_AAPL[day -1], close_data_AAPL[day -2], close_data_AAPL[day -3])
        if buy == True:
            plt.axvline(close_data_AAPL.index[day])
            for i in range(position_size):
                portfolio_trades.append(close_data_AAPL[day])
            portfolio_Balance = portfolio_Balance + close_data_AAPL[day]*position_size
            portfolio_Balance_OT.append(portfolio_Balance)
            position = position + position_size
            continue
        buy, position_size = Model(close_data_AAPL[day], close_data_AAPL[day -1], close_data_AAPL[day -2], close_data_AAPL[day -3])
        
        if close_data_AAPL[day] > close_data_AAPL[day -1] and close_data_AAPL[day -1] > close_data_AAPL[day -2]:
            position_size = 1
            plt.axvline(close_data_AAPL.index[day])
            for i in range(position_size):
                portfolio_trades.append(close_data_AAPL[day])
            portfolio_Balance = portfolio_Balance + close_data_AAPL[day]*position_size
            portfolio_Balance_OT.append(portfolio_Balance)
            position = position + position_size
            continue
            
        
            
        if close_data_AAPL[day] < close_data_AAPL[day -1] and position > 2:
            position_size = 2
            plt.axvline(close_data_AAPL.index[day])
            #portfolio_trades.append(-close_data_AAPL[day])
            portfolio_Balance = portfolio_Balance - close_data_AAPL[day]
            portfolio_Balance_OT.append(portfolio_Balance)
            position = position - position_size
            for i in range(position_size):
                profit_and_loss = profit_and_loss + (close_data_AAPL[day] - portfolio_trades.pop())
            profit_and_loss_graph.append(profit_and_loss)
            continue
            
    portfolio_Balance_OT_Frame = pd.DataFrame(portfolio_Balance_OT)
    portfolio_Balance_OT_Frame.plot()
    profit_and_loss_graph_Frame = pd.DataFrame(profit_and_loss_graph)
    profit_and_loss_graph_Frame.plot()
    print(f"Position:  {position}")
    print(f"Realised P&L:  {profit_and_loss}")

In [7]:
model = Model()
BackTest(Model=model)

NameError: name 'Model' is not defined